**This practice exercise is to help you learn Linear Regression and prepare you for the second graded exercise.**

Linear regression is simply a statistical modeling technique that helps you capture relationships between features/variales in your dataset. These are called linear, because with the increase or decrease in one or multiple features, the target variable also increases or decreases proportionally. Mathematically, it is depicted as follows:

$$y = \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + ... . \beta_{n}x_{n} + \epsilon$$

Here, 

y = target variable; the value you want to predict

$\beta$ = Coefficients. They are the weights that when combined with the feature(s), help predict the target variable

$x_{1, 2 ... n}$ = These are the features in your data. Either one of them or several of them can help predict the target variable.

$\epsilon$ = Error, because you can't model a perfect relationship. There will always be residual error.

So, when you plot this relationship, you would see the data points following a linear trend, a straight line. There would be points around that are scattered, but they will be in the minority, and they are most likely noise. 

The goal of linear regression is to find the best-fitting straight line that reduces the error between the predicted value and the actual values of the target variable. 

First, let's import all the necessary modules and libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, learning_curve, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot
import numpy as np


First, as it should always be, load the dataset. Naturally, in the graded exercise, you will accept the file path as an input from the user, and not hardcode them.  

In [ ]:
df = pd.read_csv('penguins.csv')

Now, we preprocess the data. First, let's handle the missing values, if any, using dropna().

In [ ]:
df.dropna(inplace=True) #inplace=True makes sure the changes are made to the current dataframe, and not create a new one, which is the default behaviour

Before we move any further, there are some terms one must know about:

Overfitting:
This is when your AI model learns from the data too much. Meaning, instead of figuring out relationships between different feature/features (columns) and the target variable (the value you want to predict or classify), the model simply memorizes the whole thing. What happens then is that the model can work almost perfectly on the data it was exposed to during training, but will falter when exposed to new data.

This is similar to a student simply learning only a certain set of questions, instead of understanding the answers to those questions at a conceptual level. What this means is that during exams, when that student is asked a question that they never learned, they'd perform poorly. 

This is about generalizability. One should aim for this instead of memorizing. One should train and tune the model to enable it to recognize patterns in the data and not learn them as if they are some hardcoded rigid rules. 

Underfitting:
This is, obviously, the opposite. Here, the model you're training is too simple that it does not learn the underlying patterns of the data. This could happen for many reasons like not enough data to train on, using inadequate number of features, focusing on the wrong feature or features during training, or there's just too much "noise" in the data. 

Now that you know the above two concepts, let's proceed with dividing the dataset. Typically, for linear regression, the dataset is divided into a training set and a test set, especially if your dataset is simple enough that a linear relationship exists between some feature(s) and the target variable. Just to reiterate what feature and target variable means:

Feature(s): The independent variable(s) that contributes to the dependent variable (target variable), such that a relationship between the two can be determined and depicted. In simple linear regression, there's only one such feature. For example, number of cigarettes smoked and heart rate or blood pressure, as both are known to rise during smoking. 

But typically, in real-world scenarios, multiple features contribute to the target variable, and for these cases, linear regression is extended into multiple linear regression, where instead of just one feature, multiple features are studied to find the relationship with the target variable. Multiple linear regression is more suited for addressing real-world, fairly, complex problems. 

Now, on to the training set and test set division. For simple linear regression, where the dataset does not hold any complex relationships, you'd divide your dataset into these two sets. So, training set is the data that your model is exposed to to learn from and identify patterns so that it's generalizable to unseen data, and the test set is that unseen data, which is used to evaluate how well your model has learned. Based on the outcome, you might try to fine tune some parameters, which in case of simple linear regression does not really apply. Tuning is simply tweaking some parameters/hyperparameters to help improve the model's performance. And this applies in Regularized Linear Regression, which is not covered in this exercise. You are free to explore this on your own. 

Therefore, in cases where tuning would be needed, like the Regularized Linear Regression, or in algorithms for classification, clustering, decision trees, etc., you must divide your dataset into training, validation, and test set. Here's why:

If you have just the training and test set, and you start to tune your model after your first test round, you are indirectly exposing your model to the test data as well. This runs the risk of overfitting, where your model knows only your dataset, as the "unseen" data from the test set is no longer "unseen" because of the accidental leaking of it during fine tuning of your model based on the test results. Validation set helps you avoid this pitfall. 

So, after training your model based on the training set, you "test" that model against the validation set, and then fine tune it to improve the model's performance against that validation set. Once you're happy wth the performance, you then truly test the model against the test set. This way, you do not accidentally leak the data from the test set but only the validation set, which is fine, as the test set will still remain unseen to your model. An important thing to remember is that dividing your dataset into three sets makes sense only if your dataset is large enough, like there are over 10,000 data points in your dataset. Because otherwise, you are not left with enough data for the training set to train your model with, which can lead to underfitting.   

Typically, a dataset is divided across 60-40 or 80-20, when dividing into only the training-test set. When using validation set, division is 60-20-20 or 60-30-10. But these ratios are not fixed. They depend on factors like dataset size, complexity of the model being trained, and many other factors.  

Now, let's just divide our dataset. But first determine how large the "real_estate_dataset" dataset is.

In [ ]:
# Find out the size of the dataset using the shape attribute.
print(df.shape)

You'll see that the dataset is very small, only 333 rows / data points. So, we cannot divide the dataset into three sets of training, validation, and test sets, as that would leave us with inadequate data for training. Therefore, we simply divide the dataset into training and test set. So, let's do that first.

In [ ]:
#We need sklearn.model_selection module is needed to use the train_test_split() function
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

However, the above won't work, because the division must happen only to the data without the target variable, which is the value that the model must predict. So, we must first drop that target variable from the dataset and then split whatever remains. Furthermore, since we are attempting simple linear regression, we need to identify both the predictor variable and the target variable. Predictor variable is the variable / feature/ column in your dataset that single handedly can help explain/predict the target variable. This happens only in rare cases, but unlikely in real-world contexts, where datasets have several features. There, it's possible that multiple predictors contribute to explaining/predicting the target variable. That's called multiple linear regression model. 

Anyway, proceeding with our simple linear regression model and picking the predictor (bill_length_mm) and target variable (body_mass_g) by simply asking the user:

In [ ]:
#Ask the user to provide the column that they want as the predictor and target variables

for i, col in enumerate(df.columns): #enumerate simply assigns an index to the list of columns, making it convenient for the user to select from
    print(f"{i + 1}: {col}")

predictor_idx = int(input("Enter the column number for the predictor variable: ")) - 1
predictor_col = df.columns[predictor_idx]
target_idx = int(input("Enter the column number for the target variable: ")) - 1
target_col = df.columns[target_idx]

if predictor_col == target_col:
    raise ValueError("Predictor and target columns must be different!")

X = df[[predictor_col]] # The double square brackets makes sure X is a DataFrame (2D), because otherwise it would cause an error during the fit() step
y = df[target_col]

# And now we split. Remember the sequence of the variables is important: X_train, X_test, y_train, y_test, because that's the order in which the function returns the values
#random_state below is used to make sure the split is reproducible. If you don't provide it, the split will be different every time you run the code.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


We can preprocess our data by standardizing extreme values in our numerical columns, converting categorical columns to numbers using one-hot encoding, and handling missing values in ways others than dropping them. We will come to this later. 

However, here, we are assuming the simplest scenario, where we don't need to preprocess our training data, which is the predictor variable. We know that bill_length_mm is numerical, assume that there aren't wildly extreme values to interfere with the model's training and bias our results. So, with these assumptions, let's train the model. We do this using the fit() function. This will start the training process to learn the relationship between the predictor variable (X = bill_length_mm) and the target variable (y = body_mass_g), and try to minimize the error called Mean Squared Error, so that the model is able to capture maximum amount of relationship in the training data that it is exposed to.

In [ ]:
model = LinearRegression() #create a Linear Regression model object first

model.fit(X_train, y_train) #fit the model to the training data

Now that the training is done, it's time to evaluate the model. 

We use the predict() function to evaluate the performance of the model. Here, the patterns that the model learned during the training will be used to predict the values of the target variable (y_test) for each feature row in X_test. We will also calculate some errors, which serve as metrics to gauge the performance of the model.

In [ ]:
predictions = model.predict(X_test)

#mse stands for Mean squared error, which is the average of the squared differences between the predicted and actual values
mse = mean_squared_error(y_test, predictions)

#rmse stands for Root Mean Squared Error, which is the square root of the MSE. 
# This is easier to interpret as the value is in the same units as the target variable
rmse = np.sqrt(mse)

# mae stands for Mean Absolute Error, which is the average of the absolute differences between the predicted and actual values
# This is even better than mse and rmse as it takes care of extreme values in the data and gives a better sense of the error
mae = mean_absolute_error(y_test, predictions)

# r2_score is the coefficient of determination, which is a measure of how well the model is able to predict the variation in the data
# It ranges from 0 to 1, with 1 being the best possible score
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")

#plotting the regression line
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.plot(X_test, predictions, color='red', linewidth=2, label='Regression Line')
plt.title("Regression Line")
plt.xlabel(X_test.columns[0])
plt.ylabel("Target")
plt.legend()
plt.show()

Among the errors calculated, MAE tells you on average, how much the predicted value deviates from the actual value. Furthermore, you will see that RMSE is slightly larger than MAE, which may imply that there are some errors influencing model's performance, but not too aggressively. 

Finally, R-squared of 0.18 is just too poor of a model performance. This indicates that our trained model can explain only 18% of variability in the target variable. Goal is to have it explain as much variability as possible. R-squared is between 0 - 1, with 1 being the best. 

Let's see if preprocessing the data would've helped. Since we have only the bill_length_mm as the predictor, and we have already handled missing values by simply dropping those rows, we can focus on just scaling then. While we are at it, we might as well include the logic to handle categorical variable using one-hot encoding. This converts the categorical values to numerical ones. 

Also, we need to preprocess in such a fashion that everytime the model is triggered for training, it automatically first preprocesses the data and then starts the training. 

In [ ]:
"""First, let's classify the features from the training set into two types: numeric and categorical."""
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Now we create a "pipeline" for the numeric features that scales the values so that they have a mean of 0 and standard deviation of 1
# This is important because the features may have different scales, and this can affect the performance of the model
# Because linear regression assumes that all the features are on similar scales. 
# Without scaling, features with large values (e.g. bill_length_mm) will have a larger impact on the model than features with small values, if there were any in this dataset. Scaling standardizes that.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Now we create a pipeline for the categorical features
# We use the OneHotEncoder to convert the categorical features into a format that the model can understand
# This is because linear regression can only work with numbers, and not with strings
# OneHotEncoder creates a new column for each category in the categorical feature, and assigns a 1 or 0 to each column depending on whether the category is present in the row
# For example, if there's a column with values 'A', 'B', 'C', then one-hot encoding will create three columns 'A', 'B', 'C', and assign a 1 to the column corresponding to the value in the row, and 0 to the other columns 
# The drop='first' parameter is used to drop the first column in each set of dummy variables, to avoid multicollinearity. For example, if you have a column with values 'A', 'B', 'C', then you only need two columns 'B', 'C' to represent the data, because if both 'B' and 'C' are 0, then 'A' must be 1.
# Multicollienarity is when two or more variables are highly correlated, and this can cause problems in the model 
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])

# Now we create a preprocessor using the ColumnTransformer class to apply the above two transformations to the numeric and categorical features 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# The following code creates a pipeline that first preprocesses the data and then applies the linear regression model
model = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])


#And now we repeat the process of trianing the model, but this time with the preprocessor included
model.fit(X_train, y_train) #fit the model to the training data
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")

#plotting the regression line
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.plot(X_test, predictions, color='red', linewidth=2, label='Regression Line')
plt.title("Regression Line")
plt.xlabel(X_test.columns[0])
plt.ylabel("Target")
plt.legend()
plt.show()

You will notice that the performance does not change at all. So, preprocessing didn't help.

So, let's try the approach of splitting the dataset into training, validation, and test sets. Hoping this will help improve the model's performance for the selected predictor. Remember, we are doing this, even though it's not recommended, because our dataset is very small, 333 sample size. But, we still do it for educational purposes. 

In [ ]:
#Ask the user to provide the column that they want as the predictor and target variables

for i, col in enumerate(df.columns): #enumerate simply assigns an index to the list of columns, making it convenient for the user to select from
    print(f"{i + 1}: {col}")

predictor_idx = int(input("Enter the column number for the predictor variable: ")) - 1
predictor_col = df.columns[predictor_idx]
target_idx = int(input("Enter the column number for the target variable: ")) - 1
target_col = df.columns[target_idx]

if predictor_col == target_col:
    raise ValueError("Predictor and target columns must be different!")


X = df[[predictor_col]] 
y = df[target_col]

# Split the dataset into 60% training, 20% validation, and 20% testing sets.
# Note the use of X_temp and y_temp to store the remaining 40% of the data after the first split
# It's these X_temp and y_temp that are split into validation and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])


# The training goes as before
model.fit(X_train, y_train) #fit the model to the training data

# Here's the extra step, where we first evaluate the model against the validation set
y_val_pred = model.predict(X_val)

# Then we compute the metrics on the validation set
val_mse = mean_squared_error(y_val, y_val_pred)
print(f"Validation MSE: {val_mse:.2f}")
val_rmse = np.sqrt(val_mse)
print(f"Validation RMSE: {val_rmse:.2f}")
val_mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {val_mae:.2f}")
val_r2 = r2_score(y_val, y_val_pred)
print(f"Validation R-squared: {val_r2:.2f}")


# Now we can evaluate the model on the test set
y_test_pred = model.predict(X_test)

# Then calculate all the same metrics on the test set
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test MSE: {test_mse:.2f}")
test_rmse = np.sqrt(test_mse)
print(f"Test RMSE: {test_rmse:.2f}")
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test MAE: {test_mae:.2f}")
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test R-squared: {test_r2:.2f}")

#plotting the regression line
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.plot(X_test, y_test_pred, color='red', linewidth=2, label='Regression Line')
plt.title("Regression Line")
plt.xlabel(X_test.columns[0])
plt.ylabel("Target")
plt.legend()
plt.show()

You will notice that even with validation set, model's performance, although improved, is not worth bragging about. In fact, the model performs better against validation test, but when exposed to unseen data from the test set, the model drops in performance, as R-squared drops from 0.39 to 0.22. 

So, it's become clear that preprocessing, or preprocessing and validation doesn't help much if the underlying data is small. And it may also be likely that the single predictor of bill_length_mm is really not a good predictor of body_mass_g target variable. 

But still, there's one option that we haven't explored yet, which is meant to help us in the case of small sample size. This is called K-fold cross validation. This is a fancy way of saying that the dataset is divided into K subsets (folds). The model is then trained K times, each time using K-1 folds for training, and that remaining 1 fold for validation. Then, the metrics are averaged across all the K folds to provide a more reliable evaluation of the model.

Advantage of K-fold cross validation is that every data point is utilised in the training of the model, thereby making great use of the limited data you may have. Furthermore, the averaging of the metrics across K folds removes that chance of ending up with the train-test split that is either too good (overfitting) or too bad (underfitting)

K-Fold cross validation, however, consumes more resources, as it trains the model K number of times. Bare minimum K value is usually 5. Therefore, the entire training will take place 5 times. Also, there's no train-test or train-validation-test split of the data, which means there's no "unseen" data against which the trained model is evaluated. 

Given all of the above, let's try K-fold cross validation on our small dataset. 

In [ ]:
# Prompt user for predictor and target selection
for i, col in enumerate(df.columns):  # List columns with indices for user selection
    print(f"{i + 1}: {col}")

predictor_idx = int(input("Enter the column number for the predictor variable: ")) - 1
predictor_col = df.columns[predictor_idx]
target_idx = int(input("Enter the column number for the target variable: ")) - 1
target_col = df.columns[target_idx]

# Ensure predictor and target are not the same
if predictor_col == target_col:
    raise ValueError("Predictor and target columns must be different!")

# Define predictor (X) and target (y)
X = df[[predictor_col]]  
y = df[target_col]       

# Preprocessing pipeline
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])

# K-Fold Cross Validation
k = 5

# Create KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# The default nature of cross_val_score() is that it assumes that higher values are better. But in this case, lower values (error) are better. 
# So scikit-learn, by default, negates the value that the cross_val_score() function returns.
# And since we are looking for a positive value, we negate the value again to get the actual error value
mse_scores = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)
rmse_scores = np.sqrt(mse_scores)
average_rmse = np.mean(rmse_scores)

# Mean Absolute Error (MAE)
mae_scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=kf)
average_mae = np.mean(mae_scores)


# Here, we are not negating the value because higher values are better for R-squared, which is what cross_val_score() assumes by default
r2_scores = cross_val_score(model, X, y, scoring='r2', cv=kf)
average_r2 = np.mean(r2_scores)


print(f"Average MSE across {k}-Folds: {np.mean(mse_scores):.2f}")
print(f"Average RMSE across {k}-Folds: {np.mean(rmse_scores):.2f}")
print(f"Average MAE across {k}-Folds: {np.mean(np.abs(mae_scores)):.2f}")
print(f"Average R-squared across {k}-Folds: {np.mean(r2_scores):.2f}")

Even with the recommended K-fold cross validation, the model does not improve much. All of the above tries should be clear to establish that our choice of the predictor variable is not really the right choice. You may try running the program using some other predictor, and see if the model improves significantly. There is one predictor that may perform better than others. Explore to find out which one.  

It's likely then that the dataset we have is perhaps more suitable for multiple linear regression model. 

So, let's not ask for the predictor variable. Let's just let the model training do its thing to figure out which predictors are best suited for the modeling. As mentioned earlier, a single predictor for a target variable is rare. For instance, in a dataset with features like height, sex, and weight, it is very likely that the single predictor of height would be able to explain the weight target variable to a fairly decent extent. However, if that dataset were to also include 'age', then 'height' alone would be inadequate as a predictor. One would use both height and age as predictors, which again leads us to multiple linear regression model. And this is exactly what we will attempt here.

So, instead of relying on an explicit predictor variable, let's just have only the target variable defined, whcih is necessary. Then, we simply split the dataset into training and test set, preprocess the data, and then study the metrics. Remember, we are not yet using validation set here. 

In [ ]:
# Now we ask the user to provide only the target variable

for i, col in enumerate(df.columns): #enumerate simply assigns an index to the list of columns, making it convenient for the user to select from
    print(f"{i + 1}: {col}")

target_idx = int(input("Enter the column number for the target variable: ")) - 1
target_col = df.columns[target_idx]

# We drop the target_col from the dataframe and store it in X, and the target_col is stored in y. We should also drop irrelevant columns like 'ID'
# So, now the entire X dataframe is used as the predictor variables, and y is the target variable
X = df.drop(columns=[target_col]) 
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])
 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


model = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])

model.fit(X_train, y_train) 
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


You will notice really significant improvements in the metrics. Firstly, the difference between RMSE and MAE is reduced drastically, which suggests that there are no large errors interfering with the model's performance. Finally, an R-squared value of 0.87 is very good. This means that the model we ended up with, is so good that it explain 87% of the variability in the target variable. 

Since multiple features have been utilized in producing the model, it will be interesting to see which feature contributes how much to the model. 

In [ ]:
# Retrieve the trained LinearRegression model, which is available in the pipeline, which was itself embedded in 'model'
regressor = model.named_steps['regressor']

# Create an empty list to store features we will extract from the pipeline
preprocessed_features = []

# Enter the preprocessor step of the pipeline
if 'preprocessor' in model.named_steps:
    preprocessor = model.named_steps['preprocessor']

    # Make sure the preprocessor was indeed executed / fitted
    if hasattr(preprocessor, 'transformers_'):
        for name, transformer, columns in preprocessor.transformers_:
            if name == 'num':  # Numeric features
                preprocessed_features.extend(columns) # Add the column names to the list
            elif name == 'cat':  # Categorical features
                # Check if there are any categorical features. Otherwise, this can produce an error.
                if len(columns) > 0:
                    # Retrieve OneHotEncoder from the pipeline
                    encoder = transformer.named_steps['encoder'] # Get the encoder from the transformer
                    
                    # Use fitted encoder to get feature names
                    if hasattr(encoder, 'get_feature_names_out'):
                        encoded_names = encoder.get_feature_names_out(columns) # Get the feature names from the encoder
                        preprocessed_features.extend(encoded_names) # Add the column names to the list
                    else:
                        print("Encoder is not fitted or does not support feature name retrieval.")
                else:
                    print("No categorical features to encode.")

# Retrieve the coefficients from the LinearRegression model from the pipeline
coefficients = regressor.coef_

# Map the coefficients to their corresponding feature names
feature_contributions = zip(preprocessed_features, coefficients) #zip() function combines two lists into a list of tuples. Tuple contains one element from each list, in order.

# Print feature contributions
print("\nFeature Contributions:")
for feature, coef in feature_contributions:
    print(f"{feature}: {coef:.4f}")


The coefficient values returned for each feature is the amount of change these features produce in the target variable. 

For example, bill_length_mm coefficient is 121.2697. This means that for every unit change in the value of this feature, the body_mass_gtarget variable changes by 121.2698. However, since we used StandardScaler, all the values were scaled to reflect a mean of 0 and standard deviation of 1. Therefore, whenever there's a 1 standard deviation change, NOT UNIT CHANGE, in bill_length_mm, there's a corresponding change of 121.2697 in body_mass_g. So, if for example standard deviation of the original (unscaled) bill_length_mm variable was let's say 10, then for every 10 change in this variable, there's 121.2697 change in the value of body_mass_g. 

Overall, larger coefficient values indicate larger influence of the corresponding feature on the target variable. Positive coefficients mean the change will be positive, i.e., target variable will increase with increase in the feature variable. This is positive correlation. Similarly, if the coefficient is negative, like in the case of species_Chinstap, the correlation will be negative. That means with one standard deviation increase in this predictor, body_mass_g value decreases by 299.5332. Notice how all the categrical variables have been depicted separately as their own individual predictors. This is because of one-hot encoding transformation we applied as part of the preprocessing.  

However, since we are dealing with small dataset, a straight forward train-test or even train-validation-test split is not recommended. So, let's again use the K-fold cross validation, within the multiple linear regression context. 

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

for i, col in enumerate(df.columns): 
    print(f"{i + 1}: {col}")

target_idx = int(input("Enter the column number for the target variable: ")) - 1
target_col = df.columns[target_idx]

X = df.drop(columns=[target_col]) 
y = df[target_col]

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])
 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])


X = df.drop(columns=[target_col])  # Use all columns except target as predictors
y = df[target_col]

k = 5

kf = KFold(n_splits=k, shuffle=True, random_state=42)

mse_scores = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)
rmse_scores = np.sqrt(mse_scores)
average_rmse = np.mean(rmse_scores)

mae_scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=kf)
average_mae = np.mean(mae_scores)


r2_scores = cross_val_score(model, X, y, scoring='r2', cv=kf)
average_r2 = np.mean(r2_scores)


print(f"Average MSE across {k}-Folds: {np.mean(mse_scores):.2f}")
print(f"Average RMSE across {k}-Folds: {np.mean(rmse_scores):.2f}")
print(f"Average MAE across {k}-Folds: {np.mean(np.abs(mae_scores)):.2f}")
print(f"Average R-squared across {k}-Folds: {np.mean(r2_scores):.2f}")


You will notice that R-squared stays the same, but the gap between RMSE and MAE has reduced just a tiny bit, which is a slight improvement. Moreover, K-fold cross validation is the most suitable approach to modeling based on small datasets. 

Also, unlike the previous model, we don't rely on the feature coefficients to assess model's performance. That is still possible, but K-fold cross validation is meant for a different purpose. Here, the goal is to generalize model's performance across K trainings and then use that as a metric to assess model's performance.  

Now, let's plot a learning curve, which visualizes how well your model is learning every time it's trained.

In [ ]:
import warnings

# Warnings are likely to be generated if your dataset includes categorical values. 
# This is because the preprocessing steps in the pipeline may not be able to handle all of them at every data split every time
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# Define numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

preprocessor.fit(X)  # Fit the preprocessor on the entire dataset

# Create a pipeline that includes both preprocessing and regression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Perform learning curve analysis
train_sizes, train_scores, test_scores = learning_curve(
    model, X, y, cv=kf, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calculate mean and standard deviation of scores
train_scores_mean = -train_scores.mean(axis=1)
test_scores_mean = -test_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
test_scores_std = test_scores.std(axis=1)

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, label='Training score', marker='o')
plt.plot(train_sizes, test_scores_mean, label='Validation score', marker='o')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1)
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('Negative Mean Squared Error')
plt.title('Learning Curve')
plt.legend()
plt.show()


In the produced curve, if the gap between the training error plot and validation error plot is small, then that indicates that the model is capable of generalizing well. A larger gap, where the validation error is much higher, then that would indicate overfitting. 

The validation error in our plot starts out high, because the model is being trained on small training sets, which prevents it from generalizing well. As the training size increases, the validation error decreases, ultimately almost flattening, indicating better generalizability of the model.

The somewhat flattening fo both the errors on the right suggests that this is as good as it might get. Adding more data is unlikely to improve the model. 

Similar learning curve can be plotted for all the different versions of linear regressions we've seen so far. However, their code will be different and far more verbose than the above. Regardless, their visual interpretation and importance will be the same. 

***ANNEXURE***

It is important that the target variable the user selects has only integer values. Meaning, they are of interval or ratio type. In the case they are categorical, you will have to convert the target variable values in numbers using LabelEncoder(), which basically assigns a unique number, starting from 0, to every unique categorical value. Following is the code for that. 

But what is more important to remember that if the target variable is of categorical type, then linear regression is the wrong approach. You should instead use Logistic regression. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

if y.dtype == 'object' or y.dtype.name == 'category':
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    label_encoder = encoder